In [ ]:
# για το scraping
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [ ]:
# σύνδεση με το gdrive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# βασικό url του site
main_url = 'https://www.efsyn.gr'
# αρχικό url των teaser pages
start_url = 'https://www.efsyn.gr/ellada/koinonia?page='

In [ ]:
# scraping των teasers pages για να μαζέψω τα urls των άρθρων
teasers_list = []
start_page = 2
end_page = 30

for page_num in range(start_page, end_page+1):
  page_url = start_url + str(page_num)
  response = requests.get(page_url)
  doc = BeautifulSoup(response.text, 'html.parser')

  teaser_articles_list = doc.find('div', {'class': 'default-list'}).find_all('article')

  for teaser in teaser_articles_list:
      story_dict = {}
      try:
        link = teaser.find('a')['href']
        if link.startswith("http"):
          story_dict['url'] = link
        else:
          story_dict['url'] = main_url + link
      except:
        story_dict['url'] = ''
        pass
      teasers_list.append(story_dict)

efsyn_teasers_df = pd.DataFrame(teasers_list)

In [ ]:
efsyn_teasers_df

,url
0,https://www.efsyn.gr/ellada/koinonia/400489_ne...
1,https://www.efsyn.gr/ellada/koinonia/400487_ka...
2,https://www.efsyn.gr/ellada/koinonia/400480_th...
3,https://www.efsyn.gr/ellada/koinonia/400461_tr...
4,https://www.efsyn.gr/ellada/koinonia/400450_di...
...,...
459,https://www.efsyn.gr/ellada/koinonia/397841_sp...
460,https://www.efsyn.gr/ellada/koinonia/397839_ap...
461,https://www.efsyn.gr/ellada/koinonia/397838_ex...
462,https://www.efsyn.gr/ellada/koinonia/397889_ne...


In [ ]:
# scraping των full_articles
full_articles_list = []

for article_url in efsyn_teasers_df['url']:
  response = requests.get(article_url)
  doc = BeautifulSoup(response.text, 'html.parser')
  article = doc.find('section', {'class': 'split-content__main'})
  full_article_dict = {}

  # url
  full_article_dict['url'] = article_url

  #section
  try:
    section = doc.find('a', {'class': 'default-category'}).text
    full_article_dict['section'] = section
  except:
    full_article_dict['section'] = ''
    pass

  #title
  try:
    title = article.find('h1').text
    full_article_dict['title'] = title
  except:
    full_article_dict['title'] = ''
    pass

  # date
  try:
    date = article.find('time')['datetime']
    full_article_dict['date'] = date
  except:
    full_article_dict['date'] = ''
    pass

  # author
  try:
    author = article.find('span', {'class', "article__author"}).text
    full_article_dict['author'] = author
  except:
    full_article_dict['author'] = ''
    pass

  # full_text
  try:
    p_texts_list = []
    paragraphs = article.find_all('p')
    for p in paragraphs:
      p_texts_list.append(p.text)
    full_text = ' '.join(p_texts_list)
    full_text = "".join(full_text.splitlines())
    full_article_dict['full_text'] = full_text
  except:
    full_article_dict['full_text'] = ''
    pass

  #media
  try:
    section = article.find('div', {'class': 'player-container'}) or article.find('div', {'class': 'article__media'})
    if section:
      full_article_dict['media'] = True
    else:
      full_article_dict['media'] = False
  except:
    full_article_dict['media'] = False
    pass

  full_articles_list.append(full_article_dict)
efsyn_full_articles_df = pd.DataFrame(full_articles_list)
efsyn_full_articles_df


,url,section,title,date,author,full_text,media
0,https://www.efsyn.gr/ellada/koinonia/400489_ne...,ΚΟΙΝΩΝΙΑ,Νέο σοκαριστικό βίντεο από την δολοφονική επίθ...,2023-08-10T20:40:47+03:00,efsyn.gr,Βίντεο ντοκουμέντο από τη στιγμή της δολοφονικ...,False
1,https://www.efsyn.gr/ellada/koinonia/400487_ka...,ΚΟΙΝΩΝΙΑ,Καταιγιστικός ρυθμός αποκαλύψεων μετά την τραγ...,2023-08-11T12:03:07+03:00,efsyn.gr,Άκρως αποκαλυπτικά είναι τα στοιχεία που βλέπο...,True
2,https://www.efsyn.gr/ellada/koinonia/400480_th...,ΚΟΙΝΩΝΙΑ,Θανατηφόρο τροχαίο με εγκατάλειψη στις Σέρρες,2023-08-10T18:49:56+03:00,efsyn.gr,Ένας 23χρονος μοτοσικλετιστής έχασε τη ζωή του...,False
3,https://www.efsyn.gr/ellada/koinonia/400461_tr...,ΚΟΙΝΩΝΙΑ,Τροχαία ατυχήματα και πέντε τραυματίες εξαιτία...,2023-08-10T15:00:02+03:00,efsyn.gr,Η ξαφνική αλλαγή του καιρού και η δυνατή νεροπ...,False
4,https://www.efsyn.gr/ellada/koinonia/400450_di...,ΚΟΙΝΩΝΙΑ,Διαμαρτυρία Κίνησης Πολιτών κατά ΑΠΕ για παραπ...,2023-08-10T13:52:07+03:00,efsyn.gr,Τις τελευταίες ημέρες κυκλοφόρησε μια είδηση π...,False
...,...,...,...,...,...,...,...
459,https://www.efsyn.gr/ellada/koinonia/397841_sp...,ΚΟΙΝΩΝΙΑ,Σπίτια στις φλόγες από τις μεγάλες πυρκαγιές (...,2023-07-17T19:27:09+03:00,efsyn.gr,Καίγονται σπίτια και αυτοκίνητα από τα πύρινα ...,True
460,https://www.efsyn.gr/ellada/koinonia/397839_ap...,ΚΟΙΝΩΝΙΑ,Απασφάλισε κατά Σαμαρά - Μητσοτάκη ο περιφερει...,2023-07-17T16:23:19+03:00,efsyn.gr,Άρχισαν τα όργανα στο εσωτερικό της κυβερνώσας...,False
461,https://www.efsyn.gr/ellada/koinonia/397838_ex...,ΚΟΙΝΩΝΙΑ,«Εξαιρετικά ευνοϊκές συνθήκες» για τη γρήγορη ...,2023-07-17T16:19:29+03:00,efsyn.gr,Με αφορμή τη μεγάλη πυρκαγιά που βρίσκεται σε ...,True
462,https://www.efsyn.gr/ellada/koinonia/397889_ne...,ΚΟΙΝΩΝΙΑ,Νέα διαμαρτυρία κατά των πλειστηριασμών και τω...,2023-07-17T16:00:00+03:00,efsyn.gr,Με κεντρικό σύνθημα «κανένας άνθρωπος στον δρό...,False


In [ ]:
efsyn_full_articles_df.to_csv('drive/MyDrive/efsyn.csv', index=False)